In [ ]:
import os
from urllib.parse import urljoin
import pystac
import datetime

from pystac import (
    Summaries,
)
from pystac.extensions.datacube import (
    DatacubeExtension,
    DimensionType,
    HorizontalSpatialDimensionAxis,
)

from pystac.extensions.projection import ProjectionExtension
from pystac.utils import str_to_datetime
from shapely.geometry import mapping, box


import requests
from requests.auth import HTTPBasicAuth

from dotenv import load_dotenv
load_dotenv("/home/otto/s1_zarr/.env")

True

In [3]:
username = os.getenv("username_stac_dev")
password = os.getenv("password_stac_dev")

In [18]:
print(username)
def create_collection():

    """Creation of STAC collection"""
    datacube = {
        "x": {
            "axis": HorizontalSpatialDimensionAxis.X,
            "type": DimensionType.SPATIAL,
            "extent": [20000, 720000],
        },
        "y": {
            "axis": HorizontalSpatialDimensionAxis.Y,
            "type": DimensionType.SPATIAL,
            "extent": [220000, 620000],
        },
        "time": {"type": DimensionType.TEMPORAL, "extent": ["2011-03-15T00:00:00Z", None]},
    }

    parameters = {
        "GL": {
            "description": "global radiation",
            "unit": "W m-2",
            "dimensions": ["x", "y", "time"],
        },
        "P0": {
            "description": "mean sea level pressure",
            "unit": "Pa",
            "dimensions": ["x", "y", "time"],
        },
        "RH2M": {
            "description": "relative humidity",
            "unit": "percent",
            "dimensions": ["x", "y", "time"],
        },
        "RR": {
            "description": "1-hour precipitation sum",
            "unit": "kg m-2",
            "dimensions": ["x", "y", "time"],
        },
        "T2M": {
            "description": "air temperature",
            "unit": "degree_Celsius",
            "dimensions": ["x", "y", "time"],
        },
        "TD2M": {
            "description": "dew point temperature",
            "unit": "degree_Celsius",
            "dimensions": ["x", "y", "time"],
        },
        "UU": {
            "description": "wind speed in eastward direction",
            "unit": "m s-1",
            "dimensions": ["x", "y", "time"],
        },
        "VV": {
            "description": "wind speed in northward direction",
            "unit": "m s-1",
            "dimensions": ["x", "y", "time"],
        },
    }

    collection = pystac.Collection(
        id="incal-hourly",
        title= "INCA analysis hourly data (1km)",
        description="INCA combines all available datasources - weather stations, remote sensing data, numerical weather forecast models, and a high resolution terrain model - to provide a precise analysis of the condition of the near surface atmosphere.",
        extent=pystac.Extent(
            pystac.SpatialExtent([[7.1, 45.77, 17.74, 49.48]]),
            pystac.TemporalExtent([[str_to_datetime("2011-03-15T00:00:00Z"), None]]),
        ),
        keywords=["GeoSphere", "INCA", "radiation", "pressure", "humiditiy", "precipitation", "temperature", "wind speed"],
        license="CC-BY-4.0",
        extra_fields={"cube:dimensions": datacube,
                      "cube:variables": parameters}, 
    )

    collection.providers = [
    pystac.Provider(
        name="EODC",
        roles=[
            pystac.ProviderRole.HOST,
        ],
        url="https://eodc.eu/",
        ),
    pystac.Provider(
        name="GeoSphere Austria",
        roles=[
            pystac.ProviderRole.PRODUCER,
            pystac.ProviderRole.LICENSOR,
            pystac.ProviderRole.PROCESSOR,
        ],
        url="https://data.hub.geosphere.at/",
        ),        
    ]

    collection.stac_extensions = [
        ProjectionExtension.get_schema_uri(),
        DatacubeExtension.get_schema_uri(),
    ]

    summaries_dict = {}
    summaries_dict["grid"] = ["1x1km"]
    summaries_dict["temporal resolution"] = ["hourly"]
    summaries_dict["epsg"] = ["31287"]
    summaries_dict["projection"] = ["MGI / Austria Lambert (EPSG: 31287)"]
    summaries_dict["doi"] = ["https://doi.org/10.60669/6akt-5p05"]
    summaries_dict["timezone"] = ["UTC"]
    collection.summaries = Summaries(summaries_dict)

    collection.add_asset(
    key="thumbnail",
    asset=pystac.Asset(
        href="https://raw.githubusercontent.com/eodcgmbh/s1_zarr/main/inca_thumbnail.png",  # URL or path to your thumbnail
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Thumbnail"
    ))

    bbox = [7.1, 45.77, 17.74, 49.48]
    properties = {"gsd": 1000,
                  "datetime": "2011-03-15T00:00:00Z-present",
                  "chunksize": {"x":100, "y":100, "time": 720},
                  "proj:bbox": [20000, 220000, 720000, 620000],
                  "proj:wkt2": "PROJCS[\"MGI / Austria Lambert\",GEOGCS[\"MGI\",DATUM[\"Militar_Geographische_Institute\",SPHEROID[\"Bessel 1841\",6377397.155,299.1528128,AUTHORITY[\"EPSG\",\"7004\"]],TOWGS84[577.326,90.129,463.919,5.137,1.474,5.297,2.4232],AUTHORITY[\"EPSG\",\"6312\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4312\"]],PROJECTION[\"Lambert_Conformal_Conic_2SP\"],PARAMETER[\"standard_parallel_1\",49],PARAMETER[\"standard_parallel_2\",46],PARAMETER[\"latitude_of_origin\",47.5],PARAMETER[\"central_meridian\",13.33333333333333],PARAMETER[\"false_easting\",400000],PARAMETER[\"false_northing\",400000],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AUTHORITY[\"EPSG\",\"31287\"]]",
                  "proj:shape": [701, 401],
                  "proj:geometry": mapping(box(*bbox)),
                  "proj:transform": [1000.0, 0.0, 20000.0, 0.0, 1000.0, 220000.0, 0.0, 0.0, 1.0],
                  "cube:dimensions": datacube}
    item = pystac.Item(id="zarr_store", geometry=mapping(box(*bbox)), bbox=bbox, datetime=datetime.datetime(2011,3,15), properties=properties,)
    item.add_asset("zarr_store",
                   pystac.Asset(href="https://data.eodc.eu/collections/INCA/INCA.zarr",
                    media_type=pystac.MediaType.ZARR,
                    title="INCA",
                    roles=["data"],),)            
    
    destination = "inca_stac_coll"
    collection_path = os.path.join(destination, "{}.json".format(collection.id))
    collection.set_self_href(collection_path)
    item.save_object(dest_href=os.path.join(destination, "item.json"))
    collection.save_object()
    
    return collection, item

## Create Collection
k, item = create_collection()

k.to_dict()


app_host = "https://dev.stac.eodc.eu/ingestion/v1"
url = urljoin(app_host, "/collections")

requests.post("https://dev.stac.eodc.eu/ingestion/v1/collections/incal-hourly/items", json=item.to_dict(), auth=HTTPBasicAuth(username, password), timeout = 30)
requests.put("https://dev.stac.eodc.eu/ingestion/v1/collections/incal-hourly", json=k.to_dict(), auth=HTTPBasicAuth(username, password), timeout = 30)
r = requests.post(app_host + "/collections", json=k.to_dict(), auth=HTTPBasicAuth(username, password), timeout = 30)


ingest1


In [ ]:
#requests.delete("https://dev.stac.eodc.eu/ingestion/v1/collections/incal-hourly", auth=HTTPBasicAuth(username, password), timeout = 30)

<Response [200]>

In [19]:
collection = pystac.Collection.from_file("inca_stac_coll/incal-hourly.json")

try:
    collection.validate()
    print("Collection is valid.")
except Exception as e:
    print("Validation failed:", e)

Collection is valid.


In [20]:
item = pystac.Item.from_file("inca_stac_coll/item.json")

try:
    item.validate()
    print("Item is valid.")
except Exception as e:
    print("Validation failed:", e)


Item is valid.
